In [1]:
#import libraries
import pandas as pd
import re
path = 'C:\\Users\\alhal\\Ev data\\'

In [ ]:
# #Conversions
# 1 km = 0.6213712 miles
# 1 USD = 0.8642 Euros  - 10/9/2021
# 1 USD = 0.73458 UK Pounds
# 100km/hr = 62.137 mph  x 96.56 km/h = 60 mph (1.0356)


In [2]:
#Missing Value finder
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

# Electric Vehicle Registration source

In [3]:
#Read File
df_reg = pd.read_csv(path + 'tx_ev_registrations_public.csv')
df_reg.head()

,DMV ID,DMV Snapshot (Date),ZIP Code,Vehicle Name,Registration Valid Date,Registration Expiration Date
0,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN
1,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN
2,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN
3,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN
4,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN


In [4]:
#convert to list for manipulation
zipcode = list(df_reg['ZIP Code'])

In [5]:
#format zip code
for indx, val in enumerate(zipcode):
    a = '00000' + str(val)
    a = a[-7:-2]
    zipcode[indx] = a

In [6]:
#add back to data frame
df_reg['zip_fixed'] = pd.DataFrame(zipcode)
df_reg.head()

,DMV ID,DMV Snapshot (Date),ZIP Code,Vehicle Name,Registration Valid Date,Registration Expiration Date,zip_fixed
0,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN,03106
1,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN,03106
2,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN,03106
3,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN,03106
4,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,NaN,03106


In [7]:
#missing values
missing_values_table(df_reg)

Your selected dataframe has 7 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
Registration Expiration Date,575542,100.0
ZIP Code,743,0.1


In [8]:
#Dropping column due to all values missing
df_reg = df_reg.drop(['Registration Expiration Date'], axis = 1)
df_reg.head()

,DMV ID,DMV Snapshot (Date),ZIP Code,Vehicle Name,Registration Valid Date,zip_fixed
0,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,03106
1,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,03106
2,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,03106
3,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,03106
4,1,Registration Data from DFW Clean Cities (6/3/2...,3106.0,Tesla Model S,6/3/2019,03106


In [9]:
#Dropping rows due to low percentage of missing data (0.1%)
df_reg = df_reg[(df_reg['ZIP Code'].notna())]
df_reg.count()

DMV ID                     574799
DMV Snapshot (Date)        574799
ZIP Code                   574799
Vehicle Name               574799
Registration Valid Date    574799
zip_fixed                  574799
dtype: int64

In [10]:
#Total count of registered vehicles
df_reg_total = df_reg['Vehicle Name'].count()
print("Total EV cars registred in Texas:", df_reg_total, type(df_reg_total))

Total EV cars registred in Texas: 574799 <class 'numpy.int64'>


In [11]:
#Count of registered vehicles by Vehicle Name
df_reg_model = df_reg[['Vehicle Name','zip_fixed']].groupby(['Vehicle Name']).count()
df_reg_model.head()

,zip_fixed
Vehicle Name,
Audi A3 Plug In,1569
Audi A7 Plug In,72
Audi A8 Plug In,94
Audi Q5 Plug In,2194
Audi e-tron,8572


In [12]:
#Count of registered vehicles by Zip Code
df_reg_zip = df_reg[['Vehicle Name','zip_fixed']].groupby(['zip_fixed']).count()
df_reg_zip.head()

,Vehicle Name
zip_fixed,
01040,3
02478,2
03106,29
03842,2
04243,3


# Electric Vehicles Charging Stations source

In [130]:
#convert string in list to numerical value
def read_prices(list1):
    if list1 == []:
        return None
    else:
        price = re.findall('[0-9,.]+', str(list1[0]))
        price = float(price[0])
    
    return price

In [131]:
#START HERE
df_station = pd.read_csv(path + 'alt_fuel_Stations (Oct 6 2021) (1).csv')
df_station.head()

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,ELEC,Travis Park United Methodist Church,230 E Travis St,At the church parking lot on Travis Street,San Antonio,TX,78205,NaN,210-226-8341,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELEC,BROOKFIELD PROP 300 CLAY STAT1,300 Clay St,NaN,Houston,TX,77002,NaN,888-758-4389,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ELEC,Clay Cooley Nissan,4914 S IH-35,NaN,Austin,TX,78745,NaN,512-444-1400,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ELEC,McGavock Nissan - Abilene,818 E Overland Trail,NaN,Abilene,TX,79606,NaN,325-695-2970,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELEC,McGavock Nissan - Amarillo,4700 S Soncy Rd,NaN,Amarillo,TX,79119,NaN,806-354-3550,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
df_station.shape

(2117, 65)

In [133]:
missing_values_table(df_station)

Your selected dataframe has 65 columns.
There are 45 columns that have missing values.


,Missing Values,% of Total Values
Access Days Time (French),2117,100.0
E85 Blender Pump,2117,100.0
Plus4,2117,100.0
BD Blends (French),2117,100.0
Hydrogen Is Retail,2117,100.0
CNG Dispenser Num,2117,100.0
CNG On-Site Renewable Source,2117,100.0
CNG Total Compression Capacity,2117,100.0
CNG Storage Capacity,2117,100.0
LNG On-Site Renewable Source,2117,100.0


In [134]:
#faster to keep non-null columns
df_station_cleaned = df_station[['Fuel Type Code','State','ZIP','Status Code','EV Level1 EVSE Num','EV Level2 EVSE Num','EV DC Fast Count','EV Network Web','EV Pricing','Date Last Confirmed']]
df_station_cleaned

,Fuel Type Code,State,ZIP,Status Code,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Network Web,EV Pricing,Date Last Confirmed
0,ELEC,TX,78205,E,2.0,NaN,NaN,NaN,Free,2020-09-03
1,ELEC,TX,77002,E,NaN,2.0,NaN,http://www.chargepoint.com/,NaN,2021-10-05
2,ELEC,TX,78745,E,NaN,2.0,1.0,NaN,Free,2020-06-09
3,ELEC,TX,79606,E,NaN,1.0,NaN,NaN,Free,2020-06-09
4,ELEC,TX,79119,E,NaN,2.0,NaN,NaN,Free,2020-06-09
...,...,...,...,...,...,...,...,...,...,...
2112,ELEC,TX,78723,E,NaN,2.0,NaN,http://www.chargepoint.com/,NaN,2021-10-05
2113,ELEC,TX,78723,E,NaN,2.0,NaN,http://www.chargepoint.com/,NaN,2021-10-05
2114,ELEC,TX,78723,E,NaN,2.0,NaN,http://www.chargepoint.com/,NaN,2021-10-05
2115,ELEC,TX,78214,E,NaN,1.0,NaN,NaN,NaN,NaN


In [135]:
missing_values_table(df_station_cleaned)

Your selected dataframe has 10 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values
EV Level1 EVSE Num,2095,99.0
EV DC Fast Count,1889,89.2
EV Pricing,1364,64.4
EV Level2 EVSE Num,178,8.4
EV Network Web,127,6.0
Date Last Confirmed,4,0.2


In [136]:
#df_station_cleaned[(df_station_cleaned['EV Pricing'].notna())]

In [137]:
#change null station count to 0
df_station_cleaned[['EV Level1 EVSE Num','EV Level2 EVSE Num','EV DC Fast Count']] = df_station_cleaned[['EV Level1 EVSE Num','EV Level2 EVSE Num','EV DC Fast Count']].fillna(0.) 
df_station_cleaned.head()

C:\Users\alhal\anaconda3\lib\site-packages\pandas\core\frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Fuel Type Code,State,ZIP,Status Code,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Network Web,EV Pricing,Date Last Confirmed
0,ELEC,TX,78205,E,2.0,0.0,0.0,NaN,Free,2020-09-03
1,ELEC,TX,77002,E,0.0,2.0,0.0,http://www.chargepoint.com/,NaN,2021-10-05
2,ELEC,TX,78745,E,0.0,2.0,1.0,NaN,Free,2020-06-09
3,ELEC,TX,79606,E,0.0,1.0,0.0,NaN,Free,2020-06-09
4,ELEC,TX,79119,E,0.0,2.0,0.0,NaN,Free,2020-06-09


In [138]:
price_list = list(df_station_cleaned['EV Pricing'])

In [139]:
price_set = set(price_list)
price_set

{'$0.00-$1.50/Hr Variable Parking Fee, $0.20/kWh Energy Fee',
 '$0.00-$10.00/Hr Variable Parking Fee',
 '$0.00-$10.00/Hr Variable Parking Fee, $0.39-$0.49/kWh Variable Energy Fee',
 '$0.00-$25.00/Hr Variable Parking Fee, $0.00-$0.18/kWh Variable Energy Fee',
 '$0.00-$4.00/Hr Variable Parking Fee',
 '$0.00-$5.00/Hr Variable Parking Fee',
 '$0.00-$5.00/Hr Variable Parking Fee, $0.00-$0.15/kWh Variable Energy Fee',
 '$0.18-$0.68/Hr Variable Parking Fee',
 '$0.20/kWh Energy Fee',
 '$0.28 per kWh; $0.26 per minute above 60 kW and $0.13 per minute at or below 60 kW',
 '$0.5 per hour suggested donation',
 '$0.50/Hr Parking Fee',
 '$1 per hour',
 '$1.00/Hr Parking Fee',
 '$1.00/Hr Parking Fee , $0.12/kWh Energy Fee',
 '$1.00/Hr Parking Fee , $0.20/kWh Energy Fee',
 '$1.25-$5.00/Hr Variable Parking Fee',
 '$1.35/Hr Parking Fee',
 '$1.50-$3.00/Hr Variable Parking Fee, $0.22/kWh Energy Fee',
 '$1.50/Hr Parking Fee',
 '$10 per session',
 '$2 per hour',
 '$2.00-$5.00/Hr Variable Parking Fee, $0.20/

In [49]:
price_set

{'$0.00-$1.50/Hr Variable Parking Fee, $0.20/kWh Energy Fee',
 '$0.00-$10.00/Hr Variable Parking Fee',
 '$0.00-$10.00/Hr Variable Parking Fee, $0.39-$0.49/kWh Variable Energy Fee',
 '$0.00-$25.00/Hr Variable Parking Fee, $0.00-$0.18/kWh Variable Energy Fee',
 '$0.00-$4.00/Hr Variable Parking Fee',
 '$0.00-$5.00/Hr Variable Parking Fee',
 '$0.00-$5.00/Hr Variable Parking Fee, $0.00-$0.15/kWh Variable Energy Fee',
 '$0.18-$0.68/Hr Variable Parking Fee',
 '$0.20/kWh Energy Fee',
 '$0.28 per kWh; $0.26 per minute above 60 kW and $0.13 per minute at or below 60 kW',
 '$0.5 per hour suggested donation',
 '$0.50/Hr Parking Fee',
 '$1 per hour',
 '$1.00/Hr Parking Fee',
 '$1.00/Hr Parking Fee , $0.12/kWh Energy Fee',
 '$1.00/Hr Parking Fee , $0.20/kWh Energy Fee',
 '$1.25-$5.00/Hr Variable Parking Fee',
 '$1.35/Hr Parking Fee',
 '$1.50-$3.00/Hr Variable Parking Fee, $0.22/kWh Energy Fee',
 '$1.50/Hr Parking Fee',
 '$10 per session',
 '$2 per hour',
 '$2.00-$5.00/Hr Variable Parking Fee, $0.20/

In [58]:
perhour=[]
perkWh=[]
for val in price_set:
    
    a = re.match('FREE|free|Free',str(val))
    b = re.findall('[0-9,.]+.Hr' , str(val))
    c = re.findall('[0-9,.]+ per kWh' , str(val))
    c2 = re.findall('[0-9,.]+/kWh' , str(val))
    d = re.findall('[0-9,.]+ per hour' , str(val))
    e = re.findall('[0-9,.]+ per session' , str(val))
    f = re.findall('[0-9,.]+ per month' , str(val))
    g = re.findall('[0-9,.]+ per 30 second' , str(val))
    h = re.findall('[0-9,.]+ per minute' , str(val))
    
    
    if val == None:
        perhour.append(None) 
        perkWh.append(None)
    elif a != None:
        perhour.append(0.) 
        perkWh.append(0.)
    elif b != None and c != None:
        perhour.append(read_prices(b))
        perkWh.append(read_prices(c))
    elif b != None and c2 != None:
        perhour.append(read_prices(b))
        perkWh.append(read_prices(c2))
    elif b != None:
        perhour.append(read_prices(b))
        perkWh.append(0.)
    elif d != None:
        perhour.append(read_prices(d))
        perkWh.append(0.)
    elif g != None:
        perhour.append(round(60.0*2.0*read_prices(g),3))
        perkWh.append(0.)
    elif h != None:
        perhour.append(round(60.0*read_prices(h),3))
        perkWh.append(0.)
    elif c != None:
        perhour.append(0.)
        perkWh.append(read_prices(c))
    elif c2 != None:
        perhour.append(0.)
        perkWh.append(read_prices(c2))
    else:
        perhour.append(None)
        perkWh.append(None)
#     print(val, b,d,g,h)
print(price_set)
print(perhour)        
print(perkWh)

{nan, '$1.00/Hr Parking Fee , $0.20/kWh Energy Fee', 'Level 2: $0.49 per kWh', 'FREE for first 30 minutes', '$1.50-$3.00/Hr Variable Parking Fee, $0.22/kWh Energy Fee', '$0.00-$10.00/Hr Variable Parking Fee', 'Level 2: $0.03 per minute', '$0.00-$4.00/Hr Variable Parking Fee', '$1.25-$5.00/Hr Variable Parking Fee', '$0.00-$1.50/Hr Variable Parking Fee, $0.20/kWh Energy Fee', '$1.00/Hr Parking Fee', 'Level 2: $9.99 per session', 'FREE', '$3 per hour', '$1.35/Hr Parking Fee', '$1.00/Hr Parking Fee , $0.12/kWh Energy Fee', '$4.17 per month through Austin Energy’s Plug-In EVerywhere℠ program', '$0.28 per kWh; $0.26 per minute above 60 kW and $0.13 per minute at or below 60 kW', 'Free', '$0.20/kWh Energy Fee', '$0.00-$5.00/Hr Variable Parking Fee', '$0.00-$25.00/Hr Variable Parking Fee, $0.00-$0.18/kWh Variable Energy Fee', '$10 per session', '$0.50/Hr Parking Fee', '$2.00/Hr Parking Fee', '$0.18-$0.68/Hr Variable Parking Fee', 'Level 2: $0.03 per 30 seconds', '$0.5 per hour suggested donati

In [61]:
df_set = pd.DataFrame(price_set)
df_set['perhour'] = pd.DataFrame(perhour)
df_set.head()

,0,perhour
0,NaN,NaN
1,"$1.00/Hr Parking Fee , $0.20/kWh Energy Fee",1.0
2,Level 2: $0.49 per kWh,NaN
3,FREE for first 30 minutes,0.0
4,"$1.50-$3.00/Hr Variable Parking Fee, $0.22/kWh...",3.0


In [47]:
##START HERE
price_per_kWh=[]
price_per_hour=[]
price_flat=[]
price_member=[]

for indx, val in enumerate(price_set):
    a = re.match('FREE|free|Free',str(val))
    b = re.findall('[0-9,.]+.Hr' , str(val))
    c = re.findall('[0-9,.]+ per kWh' , str(val))
    c2 = re.findall('[0-9,.]+/kWh' , str(val))
    d = re.findall('[0-9,.]+ per hour' , str(val))
    e = re.findall('[0-9,.]+ per session' , str(val))
    f = re.findall('[0-9,.]+ per month' , str(val))
    g = re.findall('[0-9,.]+ per 30 second' , str(val))
    h = re.findall('[0-9,.]+ per minute' , str(val))
    
     

    if a != None:
        price_per_kWh.append(0.)
        price_per_hour.append(0.)
        price_flat.append(0.)
        price_member.append(0.)
    elif b != None and c != None:
        price_per_kWh.append(read_prices(c))
        price_per_hour.append(read_prices(b))
        price_flat.append(0.)
        price_member.append(0.)
    elif b != None and c2 != None:
        price_per_kWh.append(read_prices(c2))
        price_per_hour.append(read_prices(b))
        price_flat.append(0.)
        price_member.append(0.)        
    elif b != None:
        price_per_kWh.append(0.)
        price_per_hour.append(read_prices(b))
        price_flat.append(0.)
        price_member.append(0.)
    elif c != None:
        price_per_kWh.append(read_prices(c))
        price_per_hour.append(0.)
        price_flat.append(0.)
        price_member.append(0.)
    elif d != None:
        price_per_kWh.append(0.)
        price_per_hour.append(read_prices(d))
        price_flat.append(0.)
        price_member.append(0.)
    elif e != None:
        price_per_kWh.append(0.)
        price_per_hour.append(0.)
        price_flat.append(read_prices(e))
        price_member.append(0.)
    elif f != None:
        price_per_kWh.append(0.)
        price_per_hour.append(0.)
        price_flat.append(0.)
        price_member.append(read_prices(f))
    elif g != None:
        price_per_kWh.append(0.)
        price_per_hour.append(round(read_prices(g) * 60.*2.,3))
        price_flat.append(0.)
        price_member.append(0.)
    elif h != None:
        price_per_kWh.append(0.)
        price_per_hour.append(round(read_prices(g) * 60,3))
        price_flat.append(0.)
        price_member.append(0.)
    else:
        price_per_kWh.append(None)
        price_per_hour.append(None)
        price_flat.append(None)
        price_member.append(None)
        
    print(val, price_per_hour[indx])
#print(set(price_per_kWh))
#print(price_per_hour)  
#print(price_flat)  
#print(price_member)  

IndexError: list index out of range

In [ ]:
print(set(price_per_kWh))
print(set(price_per_hour))
print(set(price_flat))
print(set(price_member))

In [ ]:
df_station_cleaned.iloc[693]

In [ ]:
#remove rows with missing date last confirmed (0.2%)
df_station_cleaned = df_station_cleaned[(df_station_cleaned['Date Last Confirmed'].notna())]
df_station_cleaned.count()

# Vehicle Stats from webscraped source

In [13]:
#read/split text files from webscrape
def text_to_DF (filenametype, columnname ):
    #open file
    file1 = open(path + filenametype,'r')
    x = file1.readline()   
    file1.close()
    
    #deliminate on comma
    y = x.split(',')
    
    #replace ' and [  and ]
    for indx, value in enumerate(y):
        a = value.replace("'","")
        a = a.replace('[','')
        a = a.replace(']','')
        y[indx] = a
    
    #send to dataframe
    df = pd.DataFrame(y, columns ={columnname})
    
    return df

In [14]:
df_model = text_to_DF('carmodel.txt', 'model')
df_model.head()

,model
0,Model 3 Long Range Dual Motor
1,EV6 GT
2,Enyaq iV 80
3,Spring Electric
4,Model Y Long Range Dual Motor


In [15]:
df_acceleration = text_to_DF('caracceleration.txt', 'acceleration')
df_acceleration.head()

,acceleration
0,4.4 sec
1,3.5 sec
2,8.6 sec
3,15.0 sec
4,5.0 sec


In [16]:
df_battery = text_to_DF('carbattery.txt', 'battery')
df_battery.head()

,battery
0,70
1,77.4
2,77
3,26.8
4,70


In [17]:
df_chargespeed = text_to_DF('carchargespeed.txt', 'chargespeed')
df_chargespeed.head()

,chargespeed
0,650 km/h
1,920 km/h
2,510 km/h
3,120 km/h
4,590 km/h


In [18]:
df_efficiency = text_to_DF('carefficiency.txt', 'efficiency')
df_efficiency.head()

,efficiency
0,154 Wh/km
1,196 Wh/km
2,183 Wh/km
3,158 Wh/km
4,171 Wh/km


In [19]:
df_range = text_to_DF('carrange.txt', 'range')
df_range.head()

,range
0,455 km
1,395 km
2,420 km
3,170 km
4,410 km


In [20]:
df_topspeed = text_to_DF('cartopspeed.txt', 'topspeed')
df_topspeed.head()

,topspeed
0,233 km/h
1,260 km/h
2,160 km/h
3,125 km/h
4,217 km/h


In [104]:
#open text file
file_de = open('carprice_de.txt','r')
lines = file_de.readlines()
file_de.close()

In [105]:
#convert single element list to a string
str1 = str(lines[0]) 

In [106]:
#Use ' to get the start and stop of each element
indx1=[]

for indx, val in enumerate(str1):
    if val =="'":
        indx1.append(indx)    

start1 = indx1[0:len(indx1):2]
end1 = indx1[1:len(indx1):2]        

In [107]:
#populate a list with prices in elements of a list
list1=[]
word1 = ''

for indx, val in enumerate(str1):
    
    if val != "'":
        word1 += val
    if val == "'":
        list1.append(word1)
        word1 = ''

In [108]:
#format prices from strings to float and change null to 0
list2 = []
for x in list1:
    if len(x) >=3:
        num = re.findall('[0-9]+', x)
        if len(num) == 0:
            price = 0.
        else:
            price = float(num[0])*1000+float(num[1])
        #print(num, len(num))
        list2.append(price)
        
print(list2, len(list2))        

[0.0, 65990.0, 43950.0, 20490.0, 59965.0, 29990.0, 43560.0, 98000.0, 41850.0, 43560.0, 48900.0, 35350.0, 30450.0, 135529.0, 35993.0, 47541.0, 138200.0, 0.0, 50000.0, 31990.0, 58300.0, 33850.0, 70000.0, 53560.0, 40000.0, 0.0, 47500.0, 59550.0, 34490.0, 126990.0, 106374.0, 31990.0, 39000.0, 45100.0, 48990.0, 69900.0, 34640.0, 29000.0, 38850.0, 47550.0, 58560.0, 35450.0, 39090.0, 41900.0, 0.0, 44450.0, 75000.0, 38350.0, 52500.0, 186336.0, 52850.0, 34110.0, 33800.0, 140000.0, 35460.0, 66965.0, 69100.0, 27560.0, 77300.0, 149000.0, 80000.0, 34850.0, 50415.0, 33990.0, 86990.0, 42620.0, 67300.0, 53600.0, 0.0, 47000.0, 66069.0, 59250.0, 54475.0, 25500.0, 50000.0, 49500.0, 41850.0, 29990.0, 0.0, 24790.0, 215000.0, 43000.0, 32700.0, 0.0, 77300.0, 49500.0, 0.0, 36960.0, 56216.0, 35000.0, 35000.0, 41900.0, 99800.0, 62050.0, 50000.0, 62900.0, 46900.0, 42990.0, 40000.0, 60000.0, 175000.0, 36950.0, 43433.0, 42460.0, 42600.0, 32500.0, 71388.0, 0.0, 24650.0, 23560.0, 46500.0, 106487.0, 50900.0, 55600.0,

In [111]:
#convert list of prices to dataframe
df_price_de = pd.DataFrame(list2, columns = {'priceDE'})
df_price_de.head()

,priceDE
0,0.0
1,65990.0
2,43950.0
3,20490.0
4,59965.0


In [93]:
file_uk = open('carprice_uk.txt','r')
lines = file_uk.readlines()
file_uk.close()

In [94]:
#convert single element list to a string
str1 = str(lines[0]) 

In [95]:
#Use ' to get the start and stop of each element
indx1=[]

for indx, val in enumerate(str1):
    if val =="'":
        indx1.append(indx)    

start1 = indx1[0:len(indx1):2]
end1 = indx1[1:len(indx1):2]   

In [96]:
#populate a list with prices in elements of a list
list1=[]
word1 = ''

for indx, val in enumerate(str1):
    
    if val != "'":
        word1 += val
    if val == "'":
        list1.append(word1)
        word1 = ''

In [97]:
#format prices from strings to float and change null to 0
list2 = []
for x in list1:
    if len(x) >=3:
        num = re.findall('[0-9]+', x)
        if len(num) == 0:
            price = 0.
        else:
            price = float(num[0])*1000+float(num[1])
        #print(num, len(num))
        list2.append(price)
        
print(list2, len(list2))     

[48490.0, 58295.0, 39365.0, 0.0, 54000.0, 25995.0, 40990.0, 91905.0, 0.0, 40990.0, 45145.0, 30550.0, 27225.0, 115000.0, 0.0, 43495.0, 110950.0, 40800.0, 0.0, 25995.0, 51905.0, 27660.0, 60000.0, 0.0, 35000.0, 21055.0, 44990.0, 48000.0, 26045.0, 118980.0, 95000.0, 26795.0, 31305.0, 41945.0, 48000.0, 63905.0, 30895.0, 27140.0, 32010.0, 43900.0, 59990.0, 30730.0, 32445.0, 40750.0, 71500.0, 41570.0, 68000.0, 30445.0, 45900.0, 138830.0, 52000.0, 30540.0, 0.0, 125000.0, 28435.0, 60000.0, 60600.0, 23995.0, 69905.0, 0.0, 70000.0, 0.0, 55540.0, 28795.0, 83980.0, 0.0, 59730.0, 51370.0, 0.0, 42915.0, 65720.0, 49950.0, 49980.0, 0.0, 45000.0, 42900.0, 32550.0, 0.0, 27135.0, 0.0, 189000.0, 42000.0, 28500.0, 0.0, 65195.0, 0.0, 25095.0, 29755.0, 0.0, 32500.0, 0.0, 36995.0, 79900.0, 57400.0, 46000.0, 57030.0, 41330.0, 0.0, 0.0, 50000.0, 135000.0, 32150.0, 30255.0, 38815.0, 32305.0, 26000.0, 70665.0, 31680.0, 0.0, 20495.0, 39900.0, 83580.0, 0.0, 52870.0, 26495.0, 37545.0, 90980.0, 30160.0, 0.0, 48000.0, 

In [103]:
#convert list of prices to dataframe
df_price_uk = pd.DataFrame(list2, columns = {'priceUK'})
df_price_uk.head()

,priceUK
0,48490.0
1,58295.0
2,39365.0
3,0.0
4,54000.0


In [118]:
df_car1 = pd.concat([df_model,df_acceleration, df_battery, df_chargespeed, df_efficiency, df_range, df_topspeed, df_price_de, df_price_uk], axis = 1 )
df_car1.head()

,model,acceleration,battery,chargespeed,efficiency,range,topspeed,priceDE,priceUK
0,Model 3 Long Range Dual Motor,4.4 sec,70,650 km/h,154 Wh/km,455 km,233 km/h,0.0,48490.0
1,EV6 GT,3.5 sec,77.4,920 km/h,196 Wh/km,395 km,260 km/h,65990.0,58295.0
2,Enyaq iV 80,8.6 sec,77,510 km/h,183 Wh/km,420 km,160 km/h,43950.0,39365.0
3,Spring Electric,15.0 sec,26.8,120 km/h,158 Wh/km,170 km,125 km/h,20490.0,0.0
4,Model Y Long Range Dual Motor,5.0 sec,70,590 km/h,171 Wh/km,410 km,217 km/h,59965.0,54000.0


In [119]:
df_car2 = df_car1
df_car2

,model,acceleration,battery,chargespeed,efficiency,range,topspeed,priceDE,priceUK
0,Model 3 Long Range Dual Motor,4.4 sec,70,650 km/h,154 Wh/km,455 km,233 km/h,0.0,48490.0
1,EV6 GT,3.5 sec,77.4,920 km/h,196 Wh/km,395 km,260 km/h,65990.0,58295.0
2,Enyaq iV 80,8.6 sec,77,510 km/h,183 Wh/km,420 km,160 km/h,43950.0,39365.0
3,Spring Electric,15.0 sec,26.8,120 km/h,158 Wh/km,170 km,125 km/h,20490.0,0.0
4,Model Y Long Range Dual Motor,5.0 sec,70,590 km/h,171 Wh/km,410 km,217 km/h,59965.0,54000.0
...,...,...,...,...,...,...,...,...,...
180,e-Traveller Long 50 kWh,13.1 sec,45,250 km/h,243 Wh/km,185 km,130 km/h,56690.0,49905.0
181,Zafira-e Life M 50 kWh,12.1 sec,45,240 km/h,250 Wh/km,180 km,130 km/h,53800.0,0.0
182,e-Traveller Compact 50 kWh,12.1 sec,45,250 km/h,243 Wh/km,185 km,130 km/h,50880.0,0.0
183,e-SpaceTourer XL 50 kWh,12.1 sec,45,230 km/h,257 Wh/km,175 km,130 km/h,52230.0,0.0


In [120]:
#process data and convert units 

# #Conversions
# 1 km = 0.6213712 miles
# 1 USD = 0.8642 Euros  - 10/9/2021
# 1 USD = 0.73458 UK Pounds
# 0-100km/hr = 0-62.137 mph  OR 0-96.56 km/h = 0-60 mph (1.0356)



for indx, row in df_car2.iterrows():
    #acceleration to 0-60 time in seconds
    a = df_car2.loc[indx, 'acceleration'].strip().replace(' sec','')
    a = float(a) * 1.0356
    df_car2.loc[indx, 'acceleration'] = round(a,2)
    
    #battery stays kW

    #charge speed to mph
    c = df_car2.loc[indx, 'chargespeed'].strip().replace(' km/h','')
    if c == "-":
        c = 0
    else:
        c = float(c) * 0.6213712
    df_car2.loc[indx, 'chargespeed'] = round(c,0)
    
     #efficiency to Wh/mi
    e = df_car2.loc[indx, 'efficiency'].strip().replace(' Wh/km','')
    e = float(e) / 0.6213712
    df_car2.loc[indx, 'efficiency'] = round(e,0)
    
       #range to mi
    r = df_car2.loc[indx, 'range'].strip().replace(' km','')
    r = float(r) * 0.6213712
    df_car2.loc[indx, 'range'] = round(r,0) 
    
     #topspeed to mph
    t = df_car2.loc[indx, 'topspeed'].strip().replace(' km/h','')
    t = float(t) * 0.6213712
    df_car2.loc[indx, 'topspeed'] = round(t,0)
    
    #price_de to USD
    df_car2.loc[indx,'priceDE'] = df_car2.loc[indx,'priceDE'] / 0.8642
    
#     de = df_car2.loc[indx,'priceDE']
#     de = float(de)  / 0.8642
#     df_car2.loc[indx,'priceDE'] = round(de,0)
    
    
    #price_uk to USD
    df_car2.loc[indx,'priceUK'] = df_car2.loc[indx,'priceUK'] / 0.73458
    
   
    # uk = df_car2.loc[indx,'priceUK']
    # uk = float(uk)  / 0.73458
    # df_car2.loc[indx,'priceUK'] = round(uk,0)
    
    
    
df_car2.rename(columns={"priceDE":"priceDE_inUSD", "priceUK":"priceUK_inUSD"}, inplace = True)   
df_car2.head()

,model,acceleration,battery,chargespeed,efficiency,range,topspeed,priceDE,priceUK
0,Model 3 Long Range Dual Motor,4.56,70,404.0,248.0,283.0,145.0,0.0,66011.0
1,EV6 GT,3.62,77.4,572.0,315.0,245.0,162.0,76360.0,79358.0
2,Enyaq iV 80,8.91,77,317.0,295.0,261.0,99.0,50856.0,53588.0
3,Spring Electric,15.53,26.8,75.0,254.0,106.0,78.0,23710.0,0.0
4,Model Y Long Range Dual Motor,5.18,70,367.0,275.0,255.0,135.0,69388.0,73511.0


In [126]:
df_car2['priceUSD'] = df_car2[["priceDE_inUSD", "priceUK_inUSD"]].max(axis=1)   
df_car2.head()

,model,acceleration,battery,chargespeed,efficiency,range,topspeed,priceDE_inUSD,priceUK_inUSD,priceUSD
0,Model 3 Long Range Dual Motor,4.56,70,404.0,248.0,283.0,145.0,0.0,66011.0,66011.0
1,EV6 GT,3.62,77.4,572.0,315.0,245.0,162.0,76360.0,79358.0,79358.0
2,Enyaq iV 80,8.91,77,317.0,295.0,261.0,99.0,50856.0,53588.0,53588.0
3,Spring Electric,15.53,26.8,75.0,254.0,106.0,78.0,23710.0,0.0,23710.0
4,Model Y Long Range Dual Motor,5.18,70,367.0,275.0,255.0,135.0,69388.0,73511.0,73511.0


# Merge dataframes

In [ ]:
#match car name // match stations on zip

In [148]:
set(df_car2['model'])

{' 2 Long Range Dual Motor',
 ' 2 Long Range Single Motor',
 ' 2 Standard Range Single Motor',
 ' 3 ',
 ' 3 Crossback E-Tense',
 ' 500e 3+1',
 ' 500e Cabrio',
 ' 500e Hatchback 24 kWh',
 ' 500e Hatchback 42 kWh',
 ' Air Grand Touring',
 ' Air Pure',
 ' Air Touring',
 ' Ampera-e ',
 ' Ariya 63kWh',
 ' Ariya 87kWh',
 ' Ariya e-4ORCE 63kWh',
 ' Ariya e-4ORCE 87kWh',
 ' Ariya e-4ORCE 87kWh Performance',
 ' Born 110 kW - 55 kWh',
 ' Born 150 kW - 62 kWh',
 ' Born 170 kW - 62 kWh',
 ' Born 170 kW - 82 kWh',
 ' C40 Recharge',
 ' Cooper SE ',
 ' Corsa-e ',
 ' Cybertruck Dual Motor',
 ' Cybertruck Single Motor',
 ' Cybertruck Tri Motor',
 ' EQ forfour ',
 ' EQ fortwo cabrio',
 ' EQ fortwo coupe',
 ' EQA 250',
 ' EQA 300 4MATIC',
 ' EQA 350 4MATIC',
 ' EQB 350 4MATIC',
 ' EQC 400 4MATIC',
 ' EQE 350',
 ' EQS 450+',
 ' EQS 580 4MATIC',
 ' EQS AMG 53 4MATIC+',
 ' EQV 300 Extra-Long',
 ' EQV 300 Long',
 ' EV6 GT',
 ' EV6 Long Range 2WD',
 ' EV6 Long Range AWD',
 ' EV6 Standard Range 2WD',
 ' Enyaq 

In [147]:
set(df_reg['Vehicle Name'])

{'Audi A3 Plug In',
 'Audi A7 Plug In',
 'Audi A8 Plug In',
 'Audi Q5 Plug In',
 'Audi e-tron',
 'Audi e-tron Sportback',
 'Azure Transit Connect',
 'BMW 3-Series Plug in',
 'BMW 5-Series Plug in',
 'BMW 7-Series Plug in',
 'BMW X3',
 'BMW X5',
 'BMW i3',
 'BMW i3 REx',
 'BMW i8',
 'Bentley Bentayga',
 'Cadillac CT6',
 'Cadillac ELR',
 'Chevrolet Bolt EUV',
 'Chevrolet Bolt EV',
 'Chevrolet Spark',
 'Chevrolet Volt',
 'Chrysler Pacifica',
 'E-Ride EXV',
 'Ferrari SF90 Stradale',
 'Fiat 500e',
 'Fisker Karma',
 'Ford C-Max Energi',
 'Ford Escape Plug-in Hybrid',
 'Ford Focus EV',
 'Ford Fusion Energi',
 'Ford Mustang Mach-E',
 'Ford Ranger',
 'Ford Think Neighbor',
 'GEM NEV',
 'HINO 195',
 'Honda Accord Plug In',
 'Honda Clarity BEV',
 'Honda Clarity Plug In',
 'Hyundai Ioniq EV',
 'Hyundai Ioniq Plug In',
 'Hyundai Kona Electric',
 'Hyundai Sonata Plug In',
 'Jaguar I-Pace',
 'Jeep Wrangler Plug In',
 'Karma Revero',
 'Kia Niro EV',
 'Kia Niro Plug In',
 'Kia Optima Plug In',
 'Kia So

In [140]:
df_station_cleaned.head()

,Fuel Type Code,State,ZIP,Status Code,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Network Web,EV Pricing,Date Last Confirmed
0,ELEC,TX,78205,E,2.0,0.0,0.0,NaN,Free,2020-09-03
1,ELEC,TX,77002,E,0.0,2.0,0.0,http://www.chargepoint.com/,NaN,2021-10-05
2,ELEC,TX,78745,E,0.0,2.0,1.0,NaN,Free,2020-06-09
3,ELEC,TX,79606,E,0.0,1.0,0.0,NaN,Free,2020-06-09
4,ELEC,TX,79119,E,0.0,2.0,0.0,NaN,Free,2020-06-09


In [144]:
##BROKEN HERE
df_names = df_car2[['model']].join(df_reg[['Vehicle Name']])
df_names

,model,Vehicle Name
0,Model 3 Long Range Dual Motor,Tesla Model S
1,EV6 GT,Tesla Model S
2,Enyaq iV 80,Tesla Model S
3,Spring Electric,Tesla Model S
4,Model Y Long Range Dual Motor,Tesla Model S
...,...,...
180,e-Traveller Long 50 kWh,Ford C-Max Energi
181,Zafira-e Life M 50 kWh,Ford C-Max Energi
182,e-Traveller Compact 50 kWh,Ford Fusion Energi
183,e-SpaceTourer XL 50 kWh,Ford Fusion Energi
